## DMA 2023 ##
Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`, as well as your name below:

In [209]:
NAME = "Reuel Yang"

# Lab 4: Neural Networks #
**Please read the following instructions very carefully**

## Working on the assignment / FAQs
- **Always use the seed/random_state as *42* wherever applicable** (This is to ensure repeatability in answers, across questions, students and coding environments).
- All questions will be graded manually.
- Most questions have two cells:
  - A code cell for your work/code
  - A text cell for giving your final answer
- The points each question carries are indicated.
- Most assignments have bonus questions for extra credit, do try them out!
- **Submitting the assignment** : Download the '.ipynb' and '.pdf' files from Colab and upload them to Gradescope. Do not delete any outputs from cells before submitting. Make sure to assign pages to questions when uploading your PDF to Gradescope.
- That's about it. Happy coding!


## About the dataset
This assignment uses a dataset obtained from the JSE Data Archive that contains biological and self-reported activity traits of a sample of college students at a single university uploaded in 2013. Background Information on the dataset: http://jse.amstat.org/v21n2/froelich/eyecolorgender.txt

For this lab, the dataset has already been split into a training set `df_train` and a test set `df_test`.


In [210]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [211]:
!wget http://askoski.berkeley.edu/~zp/lab_4_training.csv
!wget http://askoski.berkeley.edu/~zp/lab_4_test.csv

df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

--2023-09-27 18:42:06--  http://askoski.berkeley.edu/~zp/lab_4_training.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79177 (77K) [text/csv]
Saving to: ‘lab_4_training.csv.6’

lab_4_training.csv. 100%[===================>]  77.32K  --.-KB/s    in 0.04s   

2023-09-27 18:42:06 (2.02 MB/s) - ‘lab_4_training.csv.6’ saved [79177/79177]

--2023-09-27 18:42:06--  http://askoski.berkeley.edu/~zp/lab_4_test.csv
Resolving askoski.berkeley.edu (askoski.berkeley.edu)... 169.229.192.179
Connecting to askoski.berkeley.edu (askoski.berkeley.edu)|169.229.192.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26519 (26K) [text/csv]
Saving to: ‘lab_4_test.csv.6’

lab_4_test.csv.6    100%[===================>]  25.90K  --.-KB/s    in 0.02s   

2023-09-27 18:42:06 (1.52 MB/s) - ‘lab_4_test.csv.6’ saved [

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,577,male,20,third,hazel,72.0,180.0,0,0,5.0,No,0.0,100.0,10.0,10.0
1,677,male,19,second,hazel,72.0,120.0,1,1,16.0,Yes,9.0,70.0,3.0,5.0
2,1738,male,20,second,brown,63.0,55.0,1,2,15.0,Yes,4.5,15.0,4.0,13.0
3,1355,male,20,third,green,78.0,200.0,0,0,10.0,Yes,9.0,20.0,10.0,10.0
4,891,female,19,second,green,67.0,280.0,2,0,4.0,Yes,2.0,164.0,0.0,2.0


In [212]:
df_test.head()

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,1303,male,20,second,green,73.0,210.0,0,1,10.0,Yes,5.0,50.0,1.0,15.0
1,36,male,20,third,other,71.0,90.0,1,0,15.0,Yes,4.0,10.0,0.0,1.0
2,489,male,22,fourth,hazel,75.0,200.0,0,1,1.0,Yes,2.0,150.0,1.0,10.0
3,1415,male,19,second,brown,72.0,35.0,2,2,20.0,Yes,5.0,100.0,0.0,7.0
4,616,male,22,fourth,hazel,71.0,15.0,2,1,10.0,Yes,7.0,10.0,0.0,5.0


***
### Question 1 (1 point)###
Calculate a baseline accuracy measure using the majority class, assuming a target variable of `gender`. The majority class is the most common value of the target variable in a particular dataset. Accuracy is calculated as (true positives + true negatives) / (all negatives and positives).

**Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [213]:
# YOUR CODE HERE
majority_class = df_train['gender'].mode()[0]
predictions = np.array([majority_class] * len(df_train))
accuracy = accuracy_score(df_train['gender'], predictions)
accuracy

0.5427852348993288

**Answer: 0.5427852348993288**

**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [214]:
# YOUR CODE HERE
test_pred = np.array([majority_class] * len(df_test))
test_acc = accuracy_score(df_test['gender'], test_pred)
test_acc

0.5226130653266332

**Answer: 0.5226130653266332**

***
### Question 2 (1.5 points)###
Get started with Neural Networks.

   
Choose a NN implementation (we recommend Sklearn MLPclassifier) and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (`sigmoid`/`logistic`) function as the transfer function (another word for activation function) and use Levenberg-Marquardt backpropagation (`lbfgs`). It is possible to specify logistic in Sklearn MLPclassifier.  

**My NN implementation of choice: Sklearn MLPclassifier**

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the `height` feature of the dataset to predict the `gender`. You will have to change `gender` to a 0 and 1 class. After training, use your trained model to predict the class (`gender`) using the `height` feature from the training set. What is the accuracy of this prediction?

In [215]:
# YOUR CODE HERE
copy_train = df_train.copy()
copy_train['gender'] = (df_train['gender'] == 'female').astype(int)
X = df_train[['height']]
y = copy_train['gender']
mlp = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
mlp.fit(X, y)
y_pred = mlp.predict(X)
accuracy = accuracy_score(y, y_pred)
accuracy

0.8439597315436241

**Answer: 0.8439597315436241**

**Question 2.b (0.5 points)**  
Take the trained model from question 2.a and use it to predict the test set. This can be accomplished by taking the trained model and giving it the `height` feature values from the test set. What is the accuracy of this model on the test set?

In [216]:
# YOUR CODE HERE
copy_test = df_test.copy()
X_test = copy_test[['height']]
copy_test['gender'] = (copy_test['gender'] == 'female').astype(int)
y_test = copy_test['gender']
y_pred_test = mlp.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
accuracy_test

0.8542713567839196

**Answer: 0.8542713567839196**

**Question 2.c**   
Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the `height` feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.a and 2.b with the log version or the normalized and centered version of this feature.

In [217]:
# YOUR CODE HERE
copy_train_2c = df_train.copy()
copy_test_2c = df_test.copy()
copy_train_2c['gender'] = (copy_train_2c['gender'] == 'female').astype(int)
copy_test_2c['gender'] = (copy_test_2c['gender'] == 'female').astype(int)
X_train = copy_train_2c[['height']]
y_train = copy_train_2c['gender']
X_test_1 = copy_test_2c[['height']]
y_test_1 = copy_test_2c['gender']

X_train_log = np.log(copy_train_2c[['height']])
X_test_scaled_log = np.log(copy_test_2c[['height']])
mlp_scaled = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
mlp_scaled.fit(X_train_log, y_train)
y_pred_train_log = mlp_scaled.predict(X_train_log)
y_pred_test_log = mlp_scaled.predict(X_test_scaled_log)
accuracy_train_log = accuracy_score(y_train, y_pred_train_log)
accuracy_test_log = accuracy_score(y_test_1, y_pred_test_log)
print('traine: ', accuracy_train_log)
print('test: ', accuracy_test_log)

traine:  0.834731543624161
test:  0.8316582914572864


**Answer (accuracy on training set): 0.834731543624161**

**Answer (accuracy on test set): 0.8316582914572864**

***

### Question 3 (1 point) ###
Many of the remaining features in the dataset are categorical. No ML method accepts categorical features, so transform `year`, `eyecolor`, `exercise` into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these one-hot transformed features, train and predict the class of the test set. What was your accuracy using a Neural Network with a single 10 node hidden layer?

In [218]:
# YOUR CODE HERE
df_train3 = df_train.copy()
df_test3 = df_test.copy()
features = ['year', 'eyecolor', 'exercise']
df_train3['gender'] = (df_train3['gender'] == 'female').astype(int)
df_test3['gender'] = (df_test3['gender'] == 'female').astype(int)

encoder = OneHotEncoder(drop='first', sparse=False)
X_train_encoded = encoder.fit_transform(df_train3[features])
X_test_encoded = encoder.transform(df_test3[features])

X_train_3 = X_train_encoded
y_train_3 = df_train3['gender']

classifier = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
classifier.fit(X_train_3, y_train_3)

y_pred_train_3 = classifier.predict(X_train_encoded)
y_pred_test_3 = classifier.predict(X_test_encoded)

accuracy_train_3 = accuracy_score(df_train3['gender'], y_pred_train_3)
accuracy_test_3 = accuracy_score(df_test3['gender'], y_pred_test_3)
print('traine: ', accuracy_train_3)
print('test: ', accuracy_test_3)

traine:  0.584731543624161
test:  0.5326633165829145


**Answer: training accuracy:  0.584731543624161 //
test accuracy:  0.5326633165829145**

***
### Question 4 (3 points)###
Using a NN, report the accuracy on the test set of a model that trained only on `height` and the `eyecolor` features of instances in the training set.

**Question 4.a**  
What is the accuracy on the test set using the original `height` values (no pre-processing) and `eyecolor` as a one-hot?

In [219]:
# YOUR CODE HERE
df_train_4 = df_train.copy()
df_test_4 = df_test.copy()
df_train_4['gender'] = (df_train_4['gender'] == 'female').astype(int)
df_test_4['gender'] = (df_test_4['gender'] == 'female').astype(int)

features = ['eyecolor']

encoder4 = OneHotEncoder(drop='first', sparse=False)
X_train_encoded4 = encoder4.fit_transform(df_train_4[features])
X_test_encoded4 = encoder4.transform(df_test_4[features])

X_train_4 = X_train_encoded4
y_train_4 = df_train_4['gender']

X_train4 = pd.concat([df_train_4['height'], pd.DataFrame(X_train_encoded4)], axis=1)
X_test4 = pd.concat([df_test_4['height'], pd.DataFrame(X_test_encoded4)], axis=1)

mlp4 = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
mlp4.fit(X_train_4, y_train_4)

y_pred_train_4 = mlp4.predict(X_train_encoded4)
y_pred_test_4 = mlp4.predict(X_test_encoded4)

accuracy_train_4 = accuracy_score(df_train_4['gender'], y_pred_train_4)
accuracy_test_4 = accuracy_score(df_test_4['gender'], y_pred_test_4)
print('traine: ', accuracy_train_4)
print('test: ', accuracy_test_4)


traine:  0.5427852348993288
test:  0.5226130653266332


**Answer: training accuracy:  0.5427852348993288 //
test accuracy:  0.5226130653266332**

**Question 4.b**  
What is the accuracy on the test set using the log of `height` values (applied to both training and testing sets) and `eyecolor` as a one-hot?

In [220]:
def data_changer(train_data, test_data):
    binary = {'male': 0, 'female': 1}
    train_set = train_data.replace(binary)
    test_set = test_data.replace(binary)
    return train_set, test_set

def onehot_it(dataframe):
    object_data = dataframe.select_dtypes(include=[object])
    label_encoded_data = object_data.apply(preprocessing.LabelEncoder().fit_transform)
    encoder = preprocessing.OneHotEncoder()
    one_hot_encoded = encoder.fit_transform(label_encoded_data)
    return pd.DataFrame(one_hot_encoded.toarray())

def loggers(data):
    transformed_data = onehot_it(data)
    transformed_data.columns = transformed_data.columns.astype(str)
    transformed_data['height'] = np.log(data['height'])
    return transformed_data

train_set, test_set = data_changer(df_train, df_test)

encoded_train_data = loggers(train_set[['height', 'eyecolor']])
encoded_test_data = loggers(test_set[['height', 'eyecolor']])

Y_train_labels = train_set['gender']
classifier_encoded_log = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
classifier_encoded_log.fit(encoded_train_data, Y_train_labels)

log_predicted_train = classifier_encoded_log.predict(encoded_train_data)
train_acc = accuracy_score(Y_train_labels, log_predicted_train)

Y_test_labels = test_set['gender']
log_predicted_test = classifier_encoded_log.predict(encoded_test_data)
test_acc = accuracy_score(Y_test_labels, log_predicted_test)

print('traine: ', train_acc)
print('test: ', test_acc)

traine:  0.8406040268456376
test:  0.8567839195979899


**Answer: training accuracy:  0.8406040268456376 //
test accuracy:  0.8567839195979899**

**Question 4.c**  
What is the accuracy on the test set using the Z-score of `height` values and `eyecolor` as a one-hot?

Z-score is a normalization function. It is the value of a feature minus the average value for that feature (in the training set), divided by the standard deviation of that feature (in the training set). Remember that, whenever applying a function to a feature in the training set, it also has to be applied to that same feature in the test set.

In [221]:
# YOUR CODE HERE
def helper_4c(data):
    changed_df = onehot_it(data)
    changed_df.columns = changed_df.columns.astype(str)
    changed_df['height'] = (data['height'] - data['height'].mean()) / data['height'].std()
    return changed_df

std_encoded_train_data = helper_4c(train_set[['height', 'eyecolor']])
std_encoded_test_data = helper_4c(test_set[['height', 'eyecolor']])

Y_train_labels_std = train_set['gender']
classifier_std_encoded = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)

classifier_std_encoded.fit(std_encoded_train_data, Y_train_labels_std)

Y_test_labels_std = test_set['gender']

std_predicted_train = classifier_std_encoded.predict(std_encoded_train_data)
std_predicted_test = classifier_std_encoded.predict(std_encoded_test_data)

train_acc4c = accuracy_score(Y_train_labels_std, std_predicted_train)
test_acc4c = accuracy_score(Y_test_labels_std, std_predicted_test)

print('traine: ', train_acc4c)
print('test: ', test_acc4c)

traine:  0.8473154362416108
test:  0.8567839195979899


**Answer: training accuracy:  0.8473154362416108 //
test accuracy:  0.8567839195979899**

***
### Question 5 (1.5 points) ###
Repeat question 4 for `playgames` & `eyecolor`.

**Question 5.a**  \\
What is the accuracy on the test set using the original `playgames` values (no pre-processing) and `eyecolor` as a one-hot?

In [222]:
# YOUR CODE HERE
subset_train = train_set[['playgames', 'eyecolor']]
subset_test = test_set[['playgames', 'eyecolor']]

encoded_train = onehot_it(subset_train)
encoded_train.columns = encoded_train.columns.astype(str)
encoded_train['playgames'] = subset_train['playgames']

encoded_test = onehot_it(subset_test)
encoded_test.columns = encoded_test.columns.astype(str)
encoded_test['playgames'] = subset_test['playgames']

Y_train_labels = train_set['gender']
classifier = MLPClassifier(
    hidden_layer_sizes=(10,),
    activation='logistic',
    solver='lbfgs',
    random_state=42
)
classifier.fit(encoded_train, Y_train_labels)

predicted_train = classifier.predict(encoded_train)
acc_train = accuracy_score(Y_train_labels, predicted_train)

Y_test_labels = test_set['gender']
predicted_test = classifier.predict(encoded_test)
acc_test = accuracy_score(Y_test_labels, predicted_test)

print('traine: ', acc_train)
print('test: ', acc_test)


traine:  0.7088926174496645
test:  0.6733668341708543


**Answer: training accuracy:  0.7088926174496645 //
test accuracy:  0.6733668341708543**

**Question 5.b**  \\
What is the accuracy on the test set using the log of `playgames` values (applied to both training and testing sets) and `eyecolor` as a one-hot?

Note: You can drop rows that have 0 in the `playgames` column, in order to avoid -inf values when applying log.

In [223]:
# YOUR CODE HERE
train5 = train_set[['playgames', 'eyecolor', 'gender']].copy()
test5 = test_set[['playgames', 'eyecolor', 'gender']].copy()

train5['playgames'].replace(0.0, 0.01, inplace=True)
test5['playgames'].replace(0.0, 0.01, inplace=True)

encoded_train5b = onehot_it(train5[['playgames', 'eyecolor']])
encoded_train5b.columns = encoded_train5b.columns.astype(str)

encoded_test5b = onehot_it(test5[['playgames', 'eyecolor']])
encoded_test5b.columns = encoded_test5b.columns.astype(str)

classifier5b = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', random_state=42)
classifier5b.fit(encoded_train5b, train5['gender'])

predicted_train5b = classifier5b.predict(encoded_train5b)
actrain = accuracy_score(train5['gender'], predicted_train5b)

predicted_test5b = classifier5b.predict(encoded_test5b)
acacc = accuracy_score(test5['gender'], predicted_test5b)

print('training accuracy:', actrain)
print('test accuracy:', acacc)



training accuracy: 0.5427852348993288
test accuracy: 0.5226130653266332


**Answer: training accuracy: 0.5427852348993288 //
test accuracy: 0.5226130653266332**

**Question 5.c** \\
What is the accuracy on the test set using the Z-score of `playgames` values and `eyecolor` as a one-hot?

In [224]:
# YOUR CODE HERE
encoded_train_5c = onehot_it(train_set[['playgames', 'eyecolor']])
encoded_train_5c.columns = encoded_train_5c.columns.astype(str)

encoded_test_5c = onehot_it(test_set[['playgames', 'eyecolor']])
encoded_test_5c.columns = encoded_test_5c.columns.astype(str)

mean_playgames_train = train_set['playgames'].mean()
std_playgames_train = train_set['playgames'].std()

encoded_train_5c['playgames'] = (train_set['playgames'] - mean_playgames_train) / std_playgames_train
encoded_test_5c['playgames'] = (test_set['playgames'] - mean_playgames_train) / std_playgames_train

mlp_classifier = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', random_state=42)
mlp_classifier.fit(encoded_train_5c, train_set['gender'])

train_predictions = mlp_classifier.predict(encoded_train_5c)
acctrain = accuracy_score(train_set['gender'], train_predictions)

test_predictions = mlp_classifier.predict(encoded_test_5c)
acctest = accuracy_score(test_set['gender'], test_predictions)


print('training accuracy:', acctrain)
print('test accuracy:', acctest)

training accuracy: 0.7088926174496645
test accuracy: 0.6733668341708543


**Answer: training accuracy: 0.7088926174496645 //
test accuracy: 0.6733668341708543**

***
### Question 6 (2 points)###
Combine the features from question 3, 4, and 5 (`year`, `eyecolor`, `exercise`, `height`, `playgames`). For numeric features use the best normalization method from questions 4 and 5.

**Question 6.a**   
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [225]:
train_onehot = onehot_it(train_set)
train_onehot.columns = train_onehot.columns.astype(str)
train_onehot['height'] = (train_set['height'] - train_set['height'].mean()) / train_set['height'].std()
train_onehot['playgames'] = (train_set['playgames'] - train_set['playgames'].mean()) / train_set['playgames'].std()

test_onehot = onehot_it(test_set)
test_onehot.columns = test_onehot.columns.astype(str)
test_onehot['height'] = (test_set['height'] - test_set['height'].mean()) / test_set['height'].std()
test_onehot['playgames'] = (test_set['playgames'] - test_set['playgames'].mean()) / test_set['playgames'].std()

mlp_classifier = MLPClassifier(hidden_layer_sizes=(10,), activation='logistic', solver='lbfgs', random_state=42)
mlp_classifier.fit(train_onehot, train_set['gender'])

train_predictions = mlp_classifier.predict(train_onehot)
trainn = accuracy_score(train_set['gender'], train_predictions)

test_predictions = mlp_classifier.predict(test_onehot)
testt = accuracy_score(test_set['gender'], test_predictions)

print('training accuracy:', trainn)
print('test accuracy:', testt)

training accuracy: 0.9177852348993288
test accuracy: 0.7788944723618091


**Answer: training accuracy: 0.9177852348993288 //
test accuracy: 0.7788944723618091**

***
### Question 7- Bonus (1 point)###
Can you improve your test set prediction accuracy by 3% or more? See how close to that milestone of improvement you can get by modifying the hyperparameters of  Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate, the type of activation function etc.).

A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies more generally to any ML technique with tuning parameters.

Please give your new prediction accuracy on the test set, and write a paragraph in a text cell below with an explanation of your approach and evaluation metrics.


In [ ]:
# YOUR CODE HERE

**Answer: YOUR ANSWER HERE**

**Explanation: YOUR ANSWER HERE**